## 1. Data preparation and cleaning 🧹

Before training any model, we need to prepare the dataset properly.
The raw data consists of historical matches between **Chelsea** and **Liverpool**, between 1921 and 2025, including match date and the number of goals scored.

Steps in this section:
- **Load the dataset** (from CSV export of oGol: https://www.ogol.com.br/estatisticas/chelsea-liverpool/t81-t85?id_comp=0&grp=1&epoca_ini=0&page=1).
- **Select relevant columns**:
- `date` → independent variable \(x\) (converted into numerical format yyyyMMdd (yearmonthday) since the first recorded match).
- `goals_total` → dependent variable \(y\) (sum of Chelsea and Liverpool goals grouped by year).
- **Convert dates** into a numeric scale suitable for regression, group date by year.
- **Visual sanity check**: print dataset head, plot goals over time, and check for anomalies (outliers, wrong entries, etc.).

⚠️ *Note*: Using date as the only predictor is a simplification — the intent is to build a clean, minimal dataset for a first regression experiment.

### 1.1 Load the dataset 📂

We start by loading the dataset from the `data/All_Games_Chelsea_Vs_Liverpool.csv` file.  
The file contains historical matches, with columns for date, result, teams, and score.  

Steps in this subsection:

- Import libraries (`pandas`, `numpy`, `matplotlib`).
- Load the CSV file into a DataFrame.
- Inspect the dataset (`head()`, `info()`) to verify the structure.
- Parse the **Date** column as `datetime`.
- Split the **Score** column into numerical values (goals for each team).
- Create a new column `goals_total` = sum of Chelsea and Liverpool goals.

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file_path = "../data/All_Games_Chelsea_Vs_Liverpool.csv"

# Read as pipe-delimited, skip the inconsistent header line, and define proper column names
df = pd.read_csv(
    file_path,
    sep=r"\|",
    engine="python",
    header=None,
    skiprows=1,  # skip the first row "ChelseaResult|DateTime1|Score|Time2"
    names=["ChelseaResult", "Date", "Time1", "Score", "Time2"],
    dtype=str
)

# Strip leading/trailing spaces only on object (string) columns (avoid deprecated applymap)
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].str.strip()

# Convert "Date" column to datetime format
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Split "Score" column (format "a-b") into two integers
score_parsed = df["Score"].str.extract(r"^\s*(\d+)\s*-\s*(\d+)\s*$")
df[["Home_goals", "Away_goals"]] = score_parsed.astype("Int64")

# Compute total number of goals per match
df["goals_total"] = df["Home_goals"] + df["Away_goals"]

# Quick inspection of the cleaned dataset
print(df.head())
print(df.info())

# Detect problematic rows (invalid dates or scores)
bad_rows = df[df["Date"].isna() | df["Home_goals"].isna() | df["Away_goals"].isna()]
if not bad_rows.empty:
    print("\n Problematic rows detected:")
    display(bad_rows.head(10))

  ChelseaResult       Date      Time1 Score      Time2  Home_goals  \
0             E 1921-10-01  Liverpool   1-1    Chelsea           1   
1             D 1921-09-24    Chelsea   0-1  Liverpool           0   
2             D 1920-12-27  Liverpool   2-1    Chelsea           2   
3             E 1920-12-25    Chelsea   1-1  Liverpool           1   
4             V 1919-10-18  Liverpool   0-1    Chelsea           0   

   Away_goals  goals_total  
0           1            2  
1           1            1  
2           1            3  
3           1            2  
4           1            1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ChelseaResult  198 non-null    object        
 1   Date           198 non-null    datetime64[ns]
 2   Time1          198 non-null    object        
 3   Score          198 non-null    object    